In [23]:
# Define paths
import os


data_path = "../input"
test_dir = os.path.join(data_path, "test")
valid_dir = os.path.join(data_path, "valid")
train_dir = os.path.join(data_path, "train")
submission_path = "../working/submission.csv"

# Model path - adjust if your best model is saved in a different location
model_path = "/kaggle/input/train-yolo/yolo_weights/motor_detector/weights/best.pt"

model_path = "../working/yolo_model/checkpoint_best_ema.pth"

CONCENTRATION = 0.5
TRESHOLD = 0.10
NMS_IOU_THRESHOLD = 0.2

In [24]:
import glob
import re
from typing import List


def move_train_tomo_to_valid_directory(valid_dir, filenames):
    """
    Move the test images to the valid directory.
    """
    os.makedirs(valid_dir, exist_ok=True)
    for filename in os.listdir(train_dir):
        if filename in filenames:
            # Move the file to the valid directory
            os.rename(
                os.path.join(train_dir, filename), os.path.join(valid_dir, filename)
            )


def get_images_path(dir):
    tomo_ids_paths = sorted(
        glob.glob(os.path.join(dir, "**/*.jpg"), recursive=True),
    )

    total_tomos = len(tomo_ids_paths)
    print(f"Total slices: {total_tomos}")
    return tomo_ids_paths


def find_images_per_id(directory: str) -> dict:
    """
    Get the group of tomograms from the directory.
    """

    # Get all the tomograms
    images_paths = get_images_path(directory)

    # Group by id
    tomo_ids = {}
    for path in images_paths:
        # Get the id from the path
        id = re.search(r"tomo_[^_|/]+", path).group()
        images_paths_per_tomo_id: List = tomo_ids.get(id, [])
        images_paths_per_tomo_id.append(path)
        tomo_ids[id] = images_paths_per_tomo_id

    print(f"Total tomo ids: {len(tomo_ids)}")

    return tomo_ids


valid_tomo_ids = find_images_per_id("../working/yolo_dataset/images/valid")

Total slices: 792
Total tomo ids: 73


In [25]:
move_train_tomo_to_valid_directory(valid_dir, valid_tomo_ids.keys())

In [66]:
import glob
import numpy as np
import pandas as pd
from rfdetr import RFDETRBase
from PIL import Image
import torch
import torchvision.transforms.functional as F
import tqdm
from itertools import chain


def get_slice_files_from_tomo_id(tomo_id, test_dir, concentration):
    tomo_dir = os.path.join(test_dir, tomo_id)
    slice_files = sorted([f for f in os.listdir(tomo_dir) if f.endswith(".jpg")])

    # Apply CONCENTRATION to reduce the number of slices processed
    # This will process approximately CONCENTRATION fraction of all slices
    selected_indices = np.linspace(
        0, len(slice_files) - 1, int(len(slice_files) * concentration)
    )
    selected_indices = np.round(selected_indices).astype(int)
    slice_files = [slice_files[i] for i in selected_indices]
    print(f"Total slices in {tomo_id}: {len(slice_files)}")

    return slice_files


def load_images(slice_files):
    images = {f: Image.open(f).convert("RGB") for f in slice_files}
    return images


import supervision as sv


def get_center_from_2dbox(box: np.ndarray):
    x1, y1, x2, y2 = box

    # Calculate center coordinates
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2

    return x_center, y_center


def get_slice_number_from_filename(filename: str):
    # Extract the slice number from the filename
    # Assuming the filename format is "slice_1.jpg", "slice_2.jpg", etc.
    # Adjust the parsing logic based on your actual filename format
    slice_number = int(filename.split("/")[-1].split(".")[0].split("_")[-1])
    return slice_number


def make_center_predictions(detection: sv.Detections, slice_file):
    all_detections = []
    for idx, confidence in enumerate(detection.confidence):
        slice_num = get_slice_number_from_filename(slice_file)
        x_center, y_center = get_center_from_2dbox(detection.xyxy[idx])
        all_detections.append(
            {
                "z": round(slice_num),
                "y": round(y_center),
                "x": round(x_center),
                "confidence": float(confidence),
            }
        )
    return all_detections


def perform_3d_nms(detections, iou_threshold):
    """
    Perform 3D Non-Maximum Suppression on detections to merge nearby motors
    """
    if not detections:
        return []

    # Sort by confidence (highest first)
    detections = sorted(detections, key=lambda x: x["confidence"], reverse=True)

    # List to store final detections after NMS
    final_detections = []

    # Define 3D distance function
    def distance_3d(d1, d2):
        return np.sqrt(
            (d1["z"] - d2["z"]) ** 2
            + (d1["y"] - d2["y"]) ** 2
            + (d1["x"] - d2["x"]) ** 2
        )

    # Maximum distance threshold (based on box size and slice gap)
    box_size = 24  # Same as annotation box size
    distance_threshold = box_size * iou_threshold

    # Process each detection
    while detections:
        # Take the detection with highest confidence
        best_detection = detections.pop(0)
        final_detections.append(best_detection)

        # Filter out detections that are too close to the best detection
        detections = [
            d for d in detections if distance_3d(d, best_detection) > distance_threshold
        ]

    return final_detections


def format_for_submission(tomo_id, final_detections):
    # Sort detections by confidence (highest first)
    final_detections.sort(key=lambda x: x["confidence"], reverse=True)

    # If there are no detections, return NA values
    if not final_detections:
        return {
            "tomo_id": tomo_id,
            "Motor axis 0": -1,
            "Motor axis 1": -1,
            "Motor axis 2": -1,
        }

    # Take the detection with highest confidence
    best_detection = final_detections[0]

    # Return result with integer coordinates
    return {
        "tomo_id": tomo_id,
        "Motor axis 0": round(best_detection["z"]),
        "Motor axis 1": round(best_detection["y"]),
        "Motor axis 2": round(best_detection["x"]),
    }


def save_submission(submissions, submission_path):
    submission_df = pd.DataFrame(submissions)

    # Ensure proper column order
    submission_df = submission_df[
        ["tomo_id", "Motor axis 0", "Motor axis 1", "Motor axis 2"]
    ]

    # Save the submission file
    submission_df.to_csv(submission_path, index=False)

    print(f"\nSubmission complete!")
    print(f"Submission saved to: {submission_path}")

    # Display first few rows of submission
    print("\nSubmission preview:")
    print(submission_df.head())


def run_prediction_on_dir(
    directory,
    model,
    threshold=TRESHOLD,
    concentration=CONCENTRATION,
    submission_path=submission_path,
):
    images_per_id = find_images_per_id(directory)
    submissions = []
    print(images_per_id)
    for tomo_id, slice_files in images_per_id.items():
        images = load_images(slice_files)

        detections_list = []
        for image in tqdm.tqdm(images.values()):
            detection = model.predict(image, threshold=threshold)
            detections_list.append(detection)

        all_detections = chain.from_iterable(
            make_center_predictions(detection, filename)
            for detection, filename in zip(detections_list, images.keys())
        )
        final_detections = perform_3d_nms(all_detections, NMS_IOU_THRESHOLD)

        submissions.append(format_for_submission(tomo_id, final_detections))

    save_submission(submissions, submission_path)


model = RFDETRBase(
    pretrain_weights=model_path,
)

run_prediction_on_dir(valid_dir, model)

num_classes mismatch: pretrain weights has 0 classes, but your model has 90 classes
reinitializing detection head with 0 classes


Loading pretrain weights
Total slices: 23100
Total tomo ids: 73
{'tomo_00e463': ['../input/valid/tomo_00e463/slice_0000.jpg', '../input/valid/tomo_00e463/slice_0001.jpg', '../input/valid/tomo_00e463/slice_0002.jpg', '../input/valid/tomo_00e463/slice_0003.jpg', '../input/valid/tomo_00e463/slice_0004.jpg', '../input/valid/tomo_00e463/slice_0005.jpg', '../input/valid/tomo_00e463/slice_0006.jpg', '../input/valid/tomo_00e463/slice_0007.jpg', '../input/valid/tomo_00e463/slice_0008.jpg', '../input/valid/tomo_00e463/slice_0009.jpg', '../input/valid/tomo_00e463/slice_0010.jpg', '../input/valid/tomo_00e463/slice_0011.jpg', '../input/valid/tomo_00e463/slice_0012.jpg', '../input/valid/tomo_00e463/slice_0013.jpg', '../input/valid/tomo_00e463/slice_0014.jpg', '../input/valid/tomo_00e463/slice_0015.jpg', '../input/valid/tomo_00e463/slice_0016.jpg', '../input/valid/tomo_00e463/slice_0017.jpg', '../input/valid/tomo_00e463/slice_0018.jpg', '../input/valid/tomo_00e463/slice_0019.jpg', '../input/valid/tom

100%|██████████| 300/300 [00:09<00:00, 30.25it/s]


Submission complete!
Submission saved to: ../working/submission.csv

Submission preview:
       tomo_id  Motor axis 0  Motor axis 1  Motor axis 2
0  tomo_00e463           242           362           154
1  tomo_0da370            36           357           637
2  tomo_10c564            22           454           265
3  tomo_122a02           142           805           137
4  tomo_19a4fd           172           584           466


In [26]:
from enum import unique
import numpy as np
import pandas as pd
import sklearn.metrics


class ParticipantVisibleError(Exception):
    # If you want an error message to be shown to participants, you must raise the error as a ParticipantVisibleError
    # All other errors will only be shown to the competition host. This helps prevent unintentional leakage of solution data.
    pass


def distance_metric(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    thresh_ratio: float,
    min_radius: float,
):
    coordinate_cols = ["Motor axis 0", "Motor axis 1", "Motor axis 2"]
    unique_tomo_ids = solution["tomo_id"].unique()
    label_tensor = solution[coordinate_cols].values.reshape(
        len(unique_tomo_ids), -1, 1, len(coordinate_cols)
    )
    predicted_tensor = submission[coordinate_cols].values.reshape(
        len(unique_tomo_ids), 1, -1, len(coordinate_cols)
    )
    # Find the minimum euclidean distances between the true and predicted points
    solution["distance"] = np.linalg.norm(label_tensor - predicted_tensor, axis=2).min(
        axis=1
    )
    # Convert thresholds from angstroms to voxels
    solution["thresholds"] = solution["Voxel spacing"].apply(
        lambda x: (min_radius * thresh_ratio) / x
    )
    solution["predictions"] = submission["Has motor"].values
    solution.loc[
        (solution["distance"] > solution["thresholds"])
        & (solution["Has motor"] == 1)
        & (submission["Has motor"] == 1),
        "predictions",
    ] = 0
    return solution["predictions"].values


def score(
    solution: pd.DataFrame, submission: pd.DataFrame, min_radius: float, beta: float
) -> float:
    """
    Parameters:
    solution (pd.DataFrame): DataFrame containing ground truth motor positions.
    submission (pd.DataFrame): DataFrame containing predicted motor positions.

    Returns:
    float: FBeta score.

    Example
    --------
    >>> solution = pd.DataFrame({
    ...     'tomo_id': [0, 1, 2, 3],
    ...     'Motor axis 0': [-1, 250, 100, 200],
    ...     'Motor axis 1': [-1, 250, 100, 200],
    ...     'Motor axis 2': [-1, 250, 100, 200],
    ...     'Voxel spacing': [10, 10, 10, 10],
    ...     'Has motor': [0, 1, 1, 1]
    ... })
    >>> submission = pd.DataFrame({
    ...     'tomo_id': [0, 1, 2, 3],
    ...     'Motor axis 0': [100, 251, 600, -1],
    ...     'Motor axis 1': [100, 251, 600, -1],
    ...     'Motor axis 2': [100, 251, 600, -1]
    ... })
    >>> score(solution, submission, 1000, 2)
    0.3571428571428571
    """

    solution = solution.sort_values("tomo_id").reset_index(drop=True)
    submission = submission.sort_values("tomo_id").reset_index(drop=True)

    filename_equiv_array = (
        solution["tomo_id"].eq(submission["tomo_id"], fill_value=0).values
    )

    if np.sum(filename_equiv_array) != len(solution["tomo_id"]):
        raise ValueError(
            "Submitted tomo_id values do not match the sample_submission file"
        )

    submission.loc[:, "Has motor"] = 1
    # If any columns are missing an axis, it's marked with no motor
    select = (submission[["Motor axis 0", "Motor axis 1", "Motor axis 2"]] == -1).any(
        axis="columns"
    )
    submission.loc[select, "Has motor"] = 0

    cols = ["Has motor", "Motor axis 0", "Motor axis 1", "Motor axis 2"]
    assert all(col in submission.columns for col in cols)

    # Calculate a label of 0 or 1 using the 'has motor', and 'motor axis' values
    predictions = distance_metric(
        solution,
        submission,
        thresh_ratio=1.0,
        min_radius=min_radius,
    )
    s = sklearn.metrics.fbeta_score(
        solution["Has motor"].values, predictions, beta=beta
    )
    return s

In [27]:
valid_solution = pd.read_csv("../working/submission.csv")
valid_solution

,tomo_id,Motor axis 0,Motor axis 1,Motor axis 2
0,tomo_00e463,242,362,154
1,tomo_0da370,36,357,637
2,tomo_10c564,22,454,265
3,tomo_122a02,142,805,137
4,tomo_19a4fd,172,584,466
...,...,...,...,...
68,tomo_f78e91,102,473,388
69,tomo_f871ad,151,641,113
70,tomo_fb08b5,102,669,557
71,tomo_fd9357,126,363,140


In [28]:
ground_truth = pd.read_csv("../input/train_labels.csv")
ground_truth = ground_truth.loc[
    ground_truth["tomo_id"].isin(valid_solution["tomo_id"])
    & (ground_truth["Number of motors"] == 1)
]
ground_truth["Has motor"] = ground_truth["Number of motors"].apply(
    lambda x: 1 if x > 0 else 0
)
ground_truth

,row_id,tomo_id,Motor axis 0,Motor axis 1,Motor axis 2,Array shape (axis 0),Array shape (axis 1),Array shape (axis 2),Voxel spacing,Number of motors,Has motor
37,37,tomo_0da370,28.0,356.0,636.0,300,928,928,13.1,1,1
48,48,tomo_10c564,16.0,455.0,267.0,300,960,928,13.1,1,1
49,49,tomo_122a02,127.0,808.0,140.0,300,959,928,15.6,1,1
70,70,tomo_19a4fd,175.0,583.0,468.0,300,960,928,16.8,1,1
89,89,tomo_20a9ed,185.0,319.0,560.0,300,960,928,13.1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
713,713,tomo_f78e91,95.0,469.0,387.0,300,960,928,13.1,1,1
716,716,tomo_f871ad,157.0,642.0,115.0,300,960,928,13.1,1,1
722,722,tomo_fb08b5,106.0,671.0,559.0,300,959,928,15.6,1,1
731,731,tomo_fd9357,131.0,364.0,138.0,300,928,928,15.6,1,1


In [29]:
submission = valid_solution.loc[valid_solution["tomo_id"].isin(ground_truth["tomo_id"])]

solution = ground_truth

In [68]:
submission = pd.DataFrame(
    {
        "tomo_id": [
            0,
            1,
            2,
            2,
            2,
            3,
        ],
        "Motor axis 0": [100, 251, 260, 255, 600, -1],
        "Motor axis 1": [100, 251, 260, 255, 600, -1],
        "Motor axis 2": [100, 251, 260, 255, 600, -1],
    }
)

solution = pd.DataFrame(
    {
        "tomo_id": [0, 1, 2, 2, 3, 3],
        "Motor axis 0": [-1, 250, 100, 102, 200, 300],
        "Motor axis 1": [-1, 250, 100, 102, 200, 300],
        "Motor axis 2": [-1, 250, 100, 102, 200, 300],
        "Voxel spacing": [10, 10, 10, 10, 10, 10],
        "Has motor": [0, 1, 1, 1, 1, 1],
    }
)

In [ ]:
def join_submission_to_solution(
    solution: np.ndarray, submission: np.ndarray
) -> pd.DataFrame:
    solution = solution.reshape(-1, 1, 3)
    submission = submission.reshape(1, -1, 3)
    distances = np.linalg.norm(solution - submission, axis=2)

    # Find the minimum distance for each solution point
    min_distances = np.min(distances, axis=1)
    # Find the index of the closest submission point for each solution point

    closest_submissions_indices = np.argmin(distances, axis=1)

    closest_solutions_indices = np.argmin(distances, axis=0)

    

    return distances

In [91]:
coordinate_cols = ["Motor axis 0", "Motor axis 1", "Motor axis 2"]
for i, j in zip(
    solution.groupby("tomo_id")[coordinate_cols],
    submission.groupby("tomo_id")[coordinate_cols],
):
    print(join_submission_to_solution(i[1].to_numpy(), j[1].to_numpy()).shape)
    print(join_submission_to_solution(i[1].to_numpy(), j[1].to_numpy()))

(2, 1)
[[174.93713156]
 [  0.        ]]
(2, 1)
[[1.73205081]
 [0.        ]]
(12, 1)
[[277.12812921]
 [268.46787517]
 [866.02540378]
 [  0.        ]
 [  0.        ]
 [  1.        ]
 [273.6640276 ]
 [265.00377356]
 [862.56130217]
 [  1.        ]
 [  2.        ]
 [  2.        ]]
(4, 1)
[[348.14221232]
 [  0.        ]
 [521.34729308]
 [  0.        ]]


In [92]:
coordinate_cols = ["Motor axis 0", "Motor axis 1", "Motor axis 2"]
unique_tomo_ids = solution["tomo_id"].unique()
label_tensor = solution[coordinate_cols].values.reshape(
    len(unique_tomo_ids), -1, 1, len(coordinate_cols)
)
predicted_tensor = submission[coordinate_cols].values.reshape(
    len(unique_tomo_ids), 1, -1, len(coordinate_cols)
)

ValueError: cannot reshape array of size 18 into shape (4,newaxis,1,3)

In [18]:
score(
    solution,
    submission,
    min_radius=1000,
    beta=2,
)

ValueError: Submitted tomo_id values do not match the sample_submission file